# 랭체인(LangChain) MultiVector Retriever 예제
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## Reference : https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/multi_vector/

**문서당 여러 벡터를 저장하는 것**은 종종 유익할 수 있습니다. 여러 가지 사용 사례에서 이것이 유익합니다. LangChain에는 이러한 설정을 쉽게 쿼리할 수 있는 기본 **MultiVectorRetriever**가 있습니다. 문서당 여러 벡터를 생성하는 방법에는 여러가지 복잡성들이 존재합니다. 이번 예제에서는 이러한 벡터를 생성하고 MultiVectorRetriever를 사용하는 일반적인 방법을 다룹니다.








문서당 여러 벡터를 생성하는 방법에는 다음이 포함됩니다:

*   **더 작은 청크(Smaller chunks)**: 문서를 더 작은 청크로 나누고, 그것들을 임베딩합니다 (이것은 ParentDocumentRetriever입니다).
*   **요약(Summary)**: 각 문서에 대한 요약을 생성하고, 그것을 문서와 함께(또는 문서 대신) 임베딩합니다.
*   **가상 질문(Hypothetical questions)**: 각 문서가 적절하게 답변할 수 있는 가상 질문(hypothetical questions)을 생성하고, 그것들을 문서와 함께(또는 문서 대신) 임베딩합니다.


이 방법은 임베딩을 수동으로 추가하는 또 다른 방법을 가능하게 한다는 점을 주목하세요. 이는 특정 문서가 검색되도록 해야 하는 질문이나 쿼리를 명시적으로 추가할 수 있기 때문에 매우 유용하며, 더 많은 제어권을 제공합니다.

# LangChain 라이브러리 설치

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-openai langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [ ]:
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
loaders = [
    PyPDFLoader("https://snuac.snu.ac.kr/2015_snuac/wp-content/uploads/2015/07/asiabrief_3-26.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)

In [ ]:
docs

[Document(page_content='Seoul National University Asia Center1\n2023년 5월 22일, 3권 26호 (통권 116호)\n한국의 출산율은 어디가 바닥인가? \n0.78명! 2022년 대한민국의 합계출산율이다. 인구가 증가나 감소하\n지 않고 고령화도 되지 않는 이른바 안정인구(Stable Population)를 \n보유하기 위해서는 합계출산율이 인구대체수준인 2.1명에서 장기적\n으로 유지되어야 한다. 그런데 1/3 정도만 인구를 대체할 수 있는 출\n산율이 장기적으로 지속된다면 그만큼 우리나라 인구는 급속하게 감\n소하고 고령화될 것이다. \n이와 같은 인구학적 재생산(Reproduction) 붕괴는 저절로 회복되지 \n않는다. 이론들에 따르면 출산율이 0명에 이를 수도 있다. 대표적으\n로 이스터린(Easterlin, 1978)의 상대적 소득가설(Relative Income \nHypothesis)에 의하면, 부부가 충분한 자원을 가지고 있지 못한 경우 출산을 주저하게 되는데, 기대되는 상대적인 소득과 자원에 따라 \n출산율은 0명까지 감소할 가능성을 배제할 수 없다. 베커의 신가정경\n제출산력모형(New Home Economics Model of Fertility)에 따르면, \n자녀에 대한 만족은 자녀의 수가 아닌 자녀의 질을 통해 추구하는데 \n자녀  양육에는 보다 많은 시간 투입이 요구된다. 인적자본이 중시되\n는 현대사회에서 개인은 시간가치를 증가시키기 위해 자녀 양육보다 \n시간이 덜 소요되는 것들을 선호하게 된다. \n한국 초저출산의 원인구조\n한국에서 재생산 위기는 ‘결혼’에서부터 시작된다. 한국은 세계 경제 \n10위권, IT(Information Technology) 강국 등으로 알려져 있지만, \n<그림 1> 출산율 감소와 신생아 수를 넘어선 고령인구\n초저출산현상 극복과 인구구조 변화 대응\n# 이삼식    한양대학교\n한국의 합계출산율은 2022년 0.78명으로 장기간 초저출산현상 지

In [ ]:
len(docs)

5

# Smaller chunks

종종 **더 큰 정보 청크를 검색하되, 더 작은 청크를 임베딩하는 것이 유용**할 수 있습니다. 이렇게 하면 임베딩이 의미(Semantic meaning)를 가능한 한 정확하게 포착할 수 있지만, 가능한 많은 컨텍스트를 하위 처리 단계로 전달할 수 있습니다. 이것이 바로 ParentDocumentRetriever가 하는 일입니다. 여기서는 이 과정의 내부 동작을 설명합니다.

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

In [ ]:
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]
doc_ids

['3c028b45-6395-4460-a34d-8538274144a9',
 '95b0e38a-1f7d-459b-96a6-1c7520cb34bf',
 '2731ef4f-1dca-4d46-a5ab-ae35a3e4e8e6',
 '6501274a-cc47-4c22-aa68-96f036355e0e',
 '2fc133d3-3f73-4039-8f40-22b7ccb4bb97']

In [ ]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [ ]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [ ]:
sub_docs

[Document(page_content='Seoul National University Asia Center1\n2023년 5월 22일, 3권 26호 (통권 116호)\n한국의 출산율은 어디가 바닥인가? \n0.78명! 2022년 대한민국의 합계출산율이다. 인구가 증가나 감소하\n지 않고 고령화도 되지 않는 이른바 안정인구(Stable Population)를 \n보유하기 위해서는 합계출산율이 인구대체수준인 2.1명에서 장기적\n으로 유지되어야 한다. 그런데 1/3 정도만 인구를 대체할 수 있는 출\n산율이 장기적으로 지속된다면 그만큼 우리나라 인구는 급속하게 감\n소하고 고령화될 것이다. \n이와 같은 인구학적 재생산(Reproduction) 붕괴는 저절로 회복되지 \n않는다. 이론들에 따르면 출산율이 0명에 이를 수도 있다. 대표적으', metadata={'source': 'https://snuac.snu.ac.kr/2015_snuac/wp-content/uploads/2015/07/asiabrief_3-26.pdf', 'page': 0, 'doc_id': '3c028b45-6395-4460-a34d-8538274144a9'}),
 Document(page_content='으로 유지되어야 한다. 그런데 1/3 정도만 인구를 대체할 수 있는 출\n산율이 장기적으로 지속된다면 그만큼 우리나라 인구는 급속하게 감\n소하고 고령화될 것이다. \n이와 같은 인구학적 재생산(Reproduction) 붕괴는 저절로 회복되지 \n않는다. 이론들에 따르면 출산율이 0명에 이를 수도 있다. 대표적으\n로 이스터린(Easterlin, 1978)의 상대적 소득가설(Relative Income \nHypothesis)에 의하면, 부부가 충분한 자원을 가지고 있지 못한 경우 출산을 주저하게 되는데, 기대되는 상대적인 소득과 자원에 따라 \n출산율은 0명까지 감소할 가능성을 배제할 수 없다. 베커의 신가정경\n제출산력모형(New Home Economics Model of Fertil

In [ ]:
len(sub_docs)

48

In [ ]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
# Vectorstore alone retrieves the small chunks
retriever.vectorstore.similarity_search("한국 저출산의 원인이 뭐야?")[0]

Document(page_content='야 한다. 사실 한국 사회에서 만연하는 저출산 원인들은 다소 차이가 \n있을지라도 OECD 국가나 동유럽 국가들 일부도 직면하고 있다. 그\n럼에도 불구하고, 유독 한국 사회에서 합계출산율이 0명대로 치닫고 \n있는 점은 직접적인 저출산 원인들 이외의 요소가 작동하고 있는 것\n으로 설명할 수 있다. 대표적으로 단기간에 이룩하였던 고도 경제성\n장의 후유증으로 삶의 목표가 성공이나 물질적인 것을 추구하는 데 \n집중되었으며, 이를 위해 어린 시절부터 무한 경쟁사회 속에서 살아\n가면서 여유 없이 쫓기고 있다. 이로 인해 자녀 양육은 무한 경쟁의 \n연속선상에서 막대한 비용을 들여야 한다는 강박관념 때문에 행복보\n다는 부담으로 인식되고 있다. 따라서 노동 개혁을 통해 그간 관행으', metadata={'doc_id': '6f307ef0-7072-4f26-9ce7-855feafa291b', 'page': 1, 'source': 'https://snuac.snu.ac.kr/2015_snuac/wp-content/uploads/2015/07/asiabrief_3-26.pdf'})

In [ ]:
# Retriever returns larger chunks
larger_chunks = retriever.invoke("한국 저출산의 원인이 뭐야?")[0].page_content
larger_chunks

'세계를 잇다, 미래를 빚다!2전통적인 가부장적 유교문화의 영향이 지속되고 있어 법률에 따르지 \n않는 가족 형성은 인정되지 않고 있다. 따라서 출산은 법률혼을 전제\n로 발생한다. 그런데 한국 사회에서 결혼은 각종 조건을 충족시킬 때 \n가능하다. 학력, 학벌, 안정적 직장, 주거 등이 갖추어져야 결혼을 할 \n수 있고, 결혼 후에도 안정적인 가족생활이 가능하다. 그만큼 청년들\n은 ‘출산’에 앞서 ‘결혼’의 어려움을 직면하고 있다. 결혼의 관문을 통\n과하기 어려워지면서 만혼 경향이 증가하고, 결혼 자체를 기피하는 \n비혼도 증가하고 있다. \n과거와 달리 결혼과 출산 간 연계도 점차 약화되고 있다. 결혼이 반\n드시 출산으로 이어지지 않고 있는 것이다. 딩크족(Double Income \nNot Kids)과 같이 결혼생활이 자녀 중심에서 부부 중심으로 변화하\n는 경향도 있지만, 무엇보다도 자녀 양육에 비용이 발생하여 큰 부담\n으로 인식되고 있기 때문이다. 자녀 양육 비용은 의료·보육·(사)교육 \n등에 관한 서비스를 구매하기 위한 경제적 비용, 돌봄, 일-가정 양립 \n등을 위한 시간 비용, 출산·양육으로 인한 고용 등에서 기회비용 등\n을 포함한다. 이러한 미시적인 원인들은 개인 또는 부부가 출산을 결\n정하는 데 직접적인 영향을 미친다. 출산은 미시적인 원인들 뿐 아니\n라 거시적인 사회구조에 의해서도 영향을 받는다. 일자리 부족과 고\n용 불안정, 높은 주거비용, 과도한 교육 경쟁(학력주의와 학벌주의 만\n연), 양성 불평등, 사회규범 및 가치관의 변화 등이 출산에 영향을 미\n치는 거시적인 요소들이다. 미시적인 요인들 대부분 거시적 요인들\n에 의해 영향을 받는다. 예를 들어, 고용에서 학력주의와 학벌주의 만\n연은 사교육비를 증가시켜 자녀 양육의 경제적 부담을 높이고 있다. 출산율 회복에 성공한 국가로부터의 교훈\n이와 같이 저출산의 원인들이 명백하게 규명되고 있다는 점에서 저\n출산은 극복이 가능한 현상으로 간주할 수 있다. 실례로 서구 국

In [ ]:
len(larger_chunks)

1944

# Summary

요약(summary)은 청크의 내용을 더 정확하게 요약하여 더 나은 검색 결과를 얻을 수 있습니다. 여기에서는 요약을 생성하고 이를 임베딩하는 방법을 보여줍니다.

In [ ]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("아래 문서를 요약하세요.:\n\n{doc}")
    | ChatOpenAI(max_retries=0, openai_api_key=OPENAI_KEY)
    | StrOutputParser()
)

In [ ]:
docs

[Document(page_content='Seoul National University Asia Center1\n2023년 5월 22일, 3권 26호 (통권 116호)\n한국의 출산율은 어디가 바닥인가? \n0.78명! 2022년 대한민국의 합계출산율이다. 인구가 증가나 감소하\n지 않고 고령화도 되지 않는 이른바 안정인구(Stable Population)를 \n보유하기 위해서는 합계출산율이 인구대체수준인 2.1명에서 장기적\n으로 유지되어야 한다. 그런데 1/3 정도만 인구를 대체할 수 있는 출\n산율이 장기적으로 지속된다면 그만큼 우리나라 인구는 급속하게 감\n소하고 고령화될 것이다. \n이와 같은 인구학적 재생산(Reproduction) 붕괴는 저절로 회복되지 \n않는다. 이론들에 따르면 출산율이 0명에 이를 수도 있다. 대표적으\n로 이스터린(Easterlin, 1978)의 상대적 소득가설(Relative Income \nHypothesis)에 의하면, 부부가 충분한 자원을 가지고 있지 못한 경우 출산을 주저하게 되는데, 기대되는 상대적인 소득과 자원에 따라 \n출산율은 0명까지 감소할 가능성을 배제할 수 없다. 베커의 신가정경\n제출산력모형(New Home Economics Model of Fertility)에 따르면, \n자녀에 대한 만족은 자녀의 수가 아닌 자녀의 질을 통해 추구하는데 \n자녀  양육에는 보다 많은 시간 투입이 요구된다. 인적자본이 중시되\n는 현대사회에서 개인은 시간가치를 증가시키기 위해 자녀 양육보다 \n시간이 덜 소요되는 것들을 선호하게 된다. \n한국 초저출산의 원인구조\n한국에서 재생산 위기는 ‘결혼’에서부터 시작된다. 한국은 세계 경제 \n10위권, IT(Information Technology) 강국 등으로 알려져 있지만, \n<그림 1> 출산율 감소와 신생아 수를 넘어선 고령인구\n초저출산현상 극복과 인구구조 변화 대응\n# 이삼식    한양대학교\n한국의 합계출산율은 2022년 0.78명으로 장기간 초저출산현상 지

In [ ]:
summaries = chain.batch(docs, {"max_concurrency": 5})
summaries

['한국의 합계출산율이 0.78명으로 매우 낮아졌는데, 이는 안정인구를 유지하기 위해서는 출산율이 2.1명에서 장기적으로 유지되어야 한다는 것을 의미한다. 이런 인구학적 재생산 붕괴는 저절로 회복되지 않으며, 출산율이 0명에 이를 수도 있다는 이론도 있다. 한국의 초저출산현상은 결혼에서부터 시작되는데, 이를 극복하고 인구구조를 변화시키기 위해서는 복합적인 대책과 종합적인 대응이 필요하다. 출산율 회복과 인구구조 변화는 양자택일이 아닌 동시에 추구해야 할 사회 목표이다.',
 '한국 사회에서는 전통적인 유교문화가 여전히 영향을 미치고 있어 법률에 따른 가족 형성이 중요시되고 있으며, 결혼은 각종 조건을 충족해야 가능하다. 하지만 결혼과 출산 간 연계가 약화되고 있으며, 경제적 부담과 사회 구조적인 요인들이 출산을 결정하는데 영향을 미치고 있다. 한국은 저출산 대책을 추진하고 있지만 출산율은 낮아지고 있어 다양한 정책적 개혁이 필요하다. 과도한 경쟁사회를 지양하고 노동 개혁을 통해 저출산 현상을 극복하는 것이 중요하다.',
 '서울대학교 아시아 센터는 학력, 학벌, 젠더, 출신 지역에 따른 차별을 없애고 능력을 우선시하는 방향으로 고용기회와 승진 등에 대한 불합리한 차별을 해결해야 한다고 주장하고 있다. 또한 자녀 양육과 노후 대비를 충돌하지 않도록 사회구조와 관행을 개선하고, 동거 커플의 권리와 의무를 규정하는 법령을 제정해야 한다고 강조하고 있다. 또한 육아휴직 등을 통해 부모의 시간을 보다 확대할 필요가 있으며, 육아기근로시간단축제, 육아재택제도 등을 활성화하여 돌봄 공백과 경력단절을 해소해야 한다고 제안하고 있다.',
 '은 인구구조 변화에 대응하는 정책을 수립하여야 한다. 이를 위해서는 출산과 양육에 대한 직장 환경을 개선하고, 학생 수 감소에 대비하여 학교 통폐합을 최소화하고, 노동력 부족에 대비하여 외국인 노동자 유입을 고려하며, 노인인구 증가에 대비하여 사회보장제도를 개선하는 등 종합적인 정책이 필요하다. 이민자 유입에 대해서는 신중한 접근이 필요하

In [ ]:
len(summaries)

5

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_KEY))
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

In [ ]:
doc_ids = [str(uuid.uuid4()) for _ in docs]
doc_ids

['5c429cb2-0cbc-414d-acf4-38c27051a852',
 '3cb81a88-a302-40a7-9c45-5ae41aa2912b',
 'f7af4159-204b-4e38-ac14-1dbee32cc6bf',
 'a85e52f6-ace7-446f-9f29-8788634fae7a',
 '5bd5776a-2312-4632-9789-6fe315724609']

In [ ]:
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

In [ ]:
summary_docs

[Document(page_content='한국의 합계출산율이 0.78명으로 매우 낮아졌는데, 이는 안정인구를 유지하기 위해서는 출산율이 2.1명에서 장기적으로 유지되어야 한다는 것을 의미한다. 이런 인구학적 재생산 붕괴는 저절로 회복되지 않으며, 출산율이 0명에 이를 수도 있다는 이론도 있다. 한국의 초저출산현상은 결혼에서부터 시작되는데, 이를 극복하고 인구구조를 변화시키기 위해서는 복합적인 대책과 종합적인 대응이 필요하다. 출산율 회복과 인구구조 변화는 양자택일이 아닌 동시에 추구해야 할 사회 목표이다.', metadata={'doc_id': '5c429cb2-0cbc-414d-acf4-38c27051a852'}),
 Document(page_content='한국 사회에서는 전통적인 유교문화가 여전히 영향을 미치고 있어 법률에 따른 가족 형성이 중요시되고 있으며, 결혼은 각종 조건을 충족해야 가능하다. 하지만 결혼과 출산 간 연계가 약화되고 있으며, 경제적 부담과 사회 구조적인 요인들이 출산을 결정하는데 영향을 미치고 있다. 한국은 저출산 대책을 추진하고 있지만 출산율은 낮아지고 있어 다양한 정책적 개혁이 필요하다. 과도한 경쟁사회를 지양하고 노동 개혁을 통해 저출산 현상을 극복하는 것이 중요하다.', metadata={'doc_id': '3cb81a88-a302-40a7-9c45-5ae41aa2912b'}),
 Document(page_content='서울대학교 아시아 센터는 학력, 학벌, 젠더, 출신 지역에 따른 차별을 없애고 능력을 우선시하는 방향으로 고용기회와 승진 등에 대한 불합리한 차별을 해결해야 한다고 주장하고 있다. 또한 자녀 양육과 노후 대비를 충돌하지 않도록 사회구조와 관행을 개선하고, 동거 커플의 권리와 의무를 규정하는 법령을 제정해야 한다고 강조하고 있다. 또한 육아휴직 등을 통해 부모의 시간을 보다 확대할 필요가 있으며, 육아기근로시간단축제, 육아재택제도 등을 활성화하여 돌봄 공백과 경력단절을 해소해야 한다고 제안하고 있다.', me

In [ ]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
# # We can also add the original chunks to the vectorstore if we so want
# for i, doc in enumerate(docs):
#     doc.metadata[id_key] = doc_ids[i]
# retriever.vectorstore.add_documents(docs)

In [ ]:
sub_docs = vectorstore.similarity_search("한국 저출산의 원인이 뭐야?")
sub_docs

[Document(page_content='한국 사회에서는 전통적인 유교문화가 여전히 영향을 미치고 있어 법률에 따른 가족 형성이 중요시되고 있으며, 결혼은 각종 조건을 충족해야 가능하다. 하지만 결혼과 출산 간 연계가 약화되고 있으며, 경제적 부담과 사회 구조적인 요인들이 출산을 결정하는데 영향을 미치고 있다. 한국은 저출산 대책을 추진하고 있지만 출산율은 낮아지고 있어 다양한 정책적 개혁이 필요하다. 과도한 경쟁사회를 지양하고 노동 개혁을 통해 저출산 현상을 극복하는 것이 중요하다.', metadata={'doc_id': '3cb81a88-a302-40a7-9c45-5ae41aa2912b'}),
 Document(page_content='한국 사회에서는 전통적인 유교문화의 영향으로 법률에 따르지 않는 가족 형성이 인정되지 않고, 출산은 법률혼을 전제로 한다. 결혼은 학력, 학벌, 안정적 직장, 주거 등의 조건을 충족해야 가능하며, 결혼과 출산 간 연계가 약화되고 있다. 고용 부족, 고용 불안정, 주거비용 증가, 교육 경쟁, 양성 불평등 등이 출산에 영향을 미치는 거시적 요인이다. 저출산 대책은 다양한 원인을 종합적으로 해소해야 하지만, 우리나라의 대책은 미흡하며, 세 가지 근본적인 노력이 필요하다. 첫째, 과도한 경쟁사회를 지양하기 위해 노동시장을 개혁해야 한다. 이를 통해 자녀 양육 비용과 부담을 줄일 수 있다.', metadata={'doc_id': 'ea5b03de-44bc-4a0f-8431-a078e7317a48'}),
 Document(page_content='서울대학교 아시아연구소가 발행하는 아시아 브리프는 아시아의 현안을 분석하고 정책적 함의를 제시하며, 한국의 아시아 진출 전략과 메가 아시아 건설을 위한 공론장을 제공한다. 최근 저출산과 고령화 문제에 대한 대응을 위한 다양한 자료가 발행되고 있다. 이삼식을 비롯한 연구원들은 저출산과 고령화에 대한 대응 방안을 제시하고 있으며, 출판물과 논문을 통해 이 문제에 대한 연구를 활발히 진행하고 있다

In [ ]:
sub_docs[0]

Document(page_content='한국 사회에서는 전통적인 유교문화가 여전히 영향을 미치고 있어 법률에 따른 가족 형성이 중요시되고 있으며, 결혼은 각종 조건을 충족해야 가능하다. 하지만 결혼과 출산 간 연계가 약화되고 있으며, 경제적 부담과 사회 구조적인 요인들이 출산을 결정하는데 영향을 미치고 있다. 한국은 저출산 대책을 추진하고 있지만 출산율은 낮아지고 있어 다양한 정책적 개혁이 필요하다. 과도한 경쟁사회를 지양하고 노동 개혁을 통해 저출산 현상을 극복하는 것이 중요하다.', metadata={'doc_id': '3cb81a88-a302-40a7-9c45-5ae41aa2912b'})

In [ ]:
larger_chunks = retriever.invoke("한국 저출산의 원인이 뭐야?")
larger_chunks

[Document(page_content='세계를 잇다, 미래를 빚다!2전통적인 가부장적 유교문화의 영향이 지속되고 있어 법률에 따르지 \n않는 가족 형성은 인정되지 않고 있다. 따라서 출산은 법률혼을 전제\n로 발생한다. 그런데 한국 사회에서 결혼은 각종 조건을 충족시킬 때 \n가능하다. 학력, 학벌, 안정적 직장, 주거 등이 갖추어져야 결혼을 할 \n수 있고, 결혼 후에도 안정적인 가족생활이 가능하다. 그만큼 청년들\n은 ‘출산’에 앞서 ‘결혼’의 어려움을 직면하고 있다. 결혼의 관문을 통\n과하기 어려워지면서 만혼 경향이 증가하고, 결혼 자체를 기피하는 \n비혼도 증가하고 있다. \n과거와 달리 결혼과 출산 간 연계도 점차 약화되고 있다. 결혼이 반\n드시 출산으로 이어지지 않고 있는 것이다. 딩크족(Double Income \nNot Kids)과 같이 결혼생활이 자녀 중심에서 부부 중심으로 변화하\n는 경향도 있지만, 무엇보다도 자녀 양육에 비용이 발생하여 큰 부담\n으로 인식되고 있기 때문이다. 자녀 양육 비용은 의료·보육·(사)교육 \n등에 관한 서비스를 구매하기 위한 경제적 비용, 돌봄, 일-가정 양립 \n등을 위한 시간 비용, 출산·양육으로 인한 고용 등에서 기회비용 등\n을 포함한다. 이러한 미시적인 원인들은 개인 또는 부부가 출산을 결\n정하는 데 직접적인 영향을 미친다. 출산은 미시적인 원인들 뿐 아니\n라 거시적인 사회구조에 의해서도 영향을 받는다. 일자리 부족과 고\n용 불안정, 높은 주거비용, 과도한 교육 경쟁(학력주의와 학벌주의 만\n연), 양성 불평등, 사회규범 및 가치관의 변화 등이 출산에 영향을 미\n치는 거시적인 요소들이다. 미시적인 요인들 대부분 거시적 요인들\n에 의해 영향을 받는다. 예를 들어, 고용에서 학력주의와 학벌주의 만\n연은 사교육비를 증가시켜 자녀 양육의 경제적 부담을 높이고 있다. 출산율 회복에 성공한 국가로부터의 교훈\n이와 같이 저출산의 원인들이 명백하게 규명되고 있다는 점에서 저\n출산은 극복이 가능한 

In [ ]:
len(larger_chunks[0].page_content)

1944

# Hypothetical Queries

LLM을 사용하여 특정 문서에 대해 물어볼 수 있는 **가상 질문 목록(hypothetical questions)**을 생성할 수도 있습니다. 이러한 질문들은 임베딩될 수 있습니다.

In [ ]:
functions = [
    {
        "name": "hypothetical_questions",
        "description": "Generate hypothetical questions",
        "parameters": {
            "type": "object",
            "properties": {
                "questions": {
                    "type": "array",
                    "items": {"type": "string"},
                },
            },
            "required": ["questions"],
        },
    }
]

In [ ]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    {"doc": lambda x: x.page_content}
    # Only asking for 3 hypothetical questions, but this could be adjusted
    | ChatPromptTemplate.from_template(
        "다음 문서가 답할 수 있는 가상 질문 3개를 생성하십시오:\n\n{doc}"
    )
    | ChatOpenAI(max_retries=0, model="gpt-4", openai_api_key=OPENAI_KEY).bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

In [ ]:
docs[0]

Document(page_content='Seoul National University Asia Center1\n2023년 5월 22일, 3권 26호 (통권 116호)\n한국의 출산율은 어디가 바닥인가? \n0.78명! 2022년 대한민국의 합계출산율이다. 인구가 증가나 감소하\n지 않고 고령화도 되지 않는 이른바 안정인구(Stable Population)를 \n보유하기 위해서는 합계출산율이 인구대체수준인 2.1명에서 장기적\n으로 유지되어야 한다. 그런데 1/3 정도만 인구를 대체할 수 있는 출\n산율이 장기적으로 지속된다면 그만큼 우리나라 인구는 급속하게 감\n소하고 고령화될 것이다. \n이와 같은 인구학적 재생산(Reproduction) 붕괴는 저절로 회복되지 \n않는다. 이론들에 따르면 출산율이 0명에 이를 수도 있다. 대표적으\n로 이스터린(Easterlin, 1978)의 상대적 소득가설(Relative Income \nHypothesis)에 의하면, 부부가 충분한 자원을 가지고 있지 못한 경우 출산을 주저하게 되는데, 기대되는 상대적인 소득과 자원에 따라 \n출산율은 0명까지 감소할 가능성을 배제할 수 없다. 베커의 신가정경\n제출산력모형(New Home Economics Model of Fertility)에 따르면, \n자녀에 대한 만족은 자녀의 수가 아닌 자녀의 질을 통해 추구하는데 \n자녀  양육에는 보다 많은 시간 투입이 요구된다. 인적자본이 중시되\n는 현대사회에서 개인은 시간가치를 증가시키기 위해 자녀 양육보다 \n시간이 덜 소요되는 것들을 선호하게 된다. \n한국 초저출산의 원인구조\n한국에서 재생산 위기는 ‘결혼’에서부터 시작된다. 한국은 세계 경제 \n10위권, IT(Information Technology) 강국 등으로 알려져 있지만, \n<그림 1> 출산율 감소와 신생아 수를 넘어선 고령인구\n초저출산현상 극복과 인구구조 변화 대응\n# 이삼식    한양대학교\n한국의 합계출산율은 2022년 0.78명으로 장기간 초저출산현상 지속

In [ ]:
chain.invoke(docs[0])

['2022년 대한민국의 합계 출산율은 얼마인가요?',
 '인구가 안정적으로 유지되기 위해 필요한 합계출산율은 얼마인가요?',
 '한국의 초저출산 현상의 복합적인 원인은 무엇인가요?']

In [ ]:
docs

[Document(page_content='Seoul National University Asia Center1\n2023년 5월 22일, 3권 26호 (통권 116호)\n한국의 출산율은 어디가 바닥인가? \n0.78명! 2022년 대한민국의 합계출산율이다. 인구가 증가나 감소하\n지 않고 고령화도 되지 않는 이른바 안정인구(Stable Population)를 \n보유하기 위해서는 합계출산율이 인구대체수준인 2.1명에서 장기적\n으로 유지되어야 한다. 그런데 1/3 정도만 인구를 대체할 수 있는 출\n산율이 장기적으로 지속된다면 그만큼 우리나라 인구는 급속하게 감\n소하고 고령화될 것이다. \n이와 같은 인구학적 재생산(Reproduction) 붕괴는 저절로 회복되지 \n않는다. 이론들에 따르면 출산율이 0명에 이를 수도 있다. 대표적으\n로 이스터린(Easterlin, 1978)의 상대적 소득가설(Relative Income \nHypothesis)에 의하면, 부부가 충분한 자원을 가지고 있지 못한 경우 출산을 주저하게 되는데, 기대되는 상대적인 소득과 자원에 따라 \n출산율은 0명까지 감소할 가능성을 배제할 수 없다. 베커의 신가정경\n제출산력모형(New Home Economics Model of Fertility)에 따르면, \n자녀에 대한 만족은 자녀의 수가 아닌 자녀의 질을 통해 추구하는데 \n자녀  양육에는 보다 많은 시간 투입이 요구된다. 인적자본이 중시되\n는 현대사회에서 개인은 시간가치를 증가시키기 위해 자녀 양육보다 \n시간이 덜 소요되는 것들을 선호하게 된다. \n한국 초저출산의 원인구조\n한국에서 재생산 위기는 ‘결혼’에서부터 시작된다. 한국은 세계 경제 \n10위권, IT(Information Technology) 강국 등으로 알려져 있지만, \n<그림 1> 출산율 감소와 신생아 수를 넘어선 고령인구\n초저출산현상 극복과 인구구조 변화 대응\n# 이삼식    한양대학교\n한국의 합계출산율은 2022년 0.78명으로 장기간 초저출산현상 지

In [ ]:
len(docs)

5

In [ ]:
hypothetical_questions = chain.batch(docs, {"max_concurrency": 1})
hypothetical_questions

[['2022년 대한민국의 합계출산율은 얼마인가?',
  '인구가 안정적으로 유지되려면 어느 정도의 합계출산율이 필요한가?',
  '한국의 초저출산현상이 지속된다면 어떤 사회적 문제가 발생할 수 있는가?'],
 ['한국 사회에서 결혼과 출산의 관계는 어떻게 변화하고 있나요?',
  '자녀 양육에 대한 부담은 어떤 요소들로 인해 발생하나요?',
  '저출산 현상을 극복하기 위한 대책은 무엇인가요?'],
 ['서울대 아시아 센터3에서는 어떤 불합리한 차별을 없애기 위한 방안을 제시하고 있나요?',
  '주택가격을 안정화하고, 초장기 저리 공공주택의 공급을 늘리는 이유는 무엇인가요?',
  '자녀 양육 지원은 어떠한 방식으로 제공되어야 한다고 서울대 아시아 센터3에서는 주장하고 있나요?'],
 ['우리나라는 얼마나 오랫동안 초저출산현상을 겪고 있나요?',
  '저출산 현상으로 인해 어떤 부분들이 파장을 받아야 하나요?',
  '노동력 부족에 대응하기 위해 어떤 전략들이 필요한가요?'],
 ['서울대학교 아시아연구소에서 발행하는 아시아 브리프의 목표는 무엇인가요?',
  '이삼식 교수는 어떤 직위를 가지고 있나요?',
  '출산율 회복과 인구구조 변화 대응 중 어느 것을 우선해야 하는가?']]

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="hypo-questions", embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

In [ ]:
doc_ids = [str(uuid.uuid4()) for _ in docs]
doc_ids

['9d2ac0d3-0f90-4012-bb88-ed194fb4e4cb',
 '37444758-5309-4f13-a4c1-d4d7482932ad',
 '078bd8ad-62a1-441f-a79d-ab035528ef64',
 '339779dd-81a5-4d00-b495-de6cdb490103',
 '75a1e36b-5f66-4668-ac40-d63e4c15677b']

In [ ]:
docs[3]

Document(page_content='세계를 잇다, 미래를 빚다!4지대를 없애도록 한다. 무엇보다도 출산과 양육에 친화적인 직장문\n화를 조성하여야 한다. \n인구구조 변화에 대응해야\n우리나라는 이미 20년 이상 동안 초저출산현상(합계출산율 1.3명 \n이하)을 겪고 있다. 적어도 당분간 ‘저출산의 덫’에서 빠져나올 수 있\n을 것으로 기대하기 어렵다. 저출산 대책들이 실효성을 거두기 시작\n할지라도 언제부터 얼마나 빠른 속도로 어느 수준까지 출산율이 높\n아질 것으로 예상하기도 어렵다. 결국 우리나라 인구는 급속하게 감\n소하고, 고령화될 것이다. 따라서 저출산 대책을 더욱 강화하는 동시\n에 인구감소 및 고령화에 대해서도 철저하게 대응하여야 한다. 인구\n감소 및 고령화가 사회에 미치는 파장은 크게 학교 붕괴, 병력자원 부\n족, 노동력 부족, 사회보장 부담 증가 등을 들 수 있다. \n학령인구 감소에 따른 학생 수 감소는 초등학교, 중학교, 고등학교 및 \n대학교에까지 영향을 미치고 있다. 과거 고출산 시기에 태어난 세대\n를 위해 건립하였던 학교 시설과 교사 양성 인프라(교육대학 등)는 과\n잉 시설이 되면서 통폐합이 진행되고 있다. 폐교는 단순하게 잉여 시\n설의 처분만을 의미하지 않는다. 폐교로 아동이 있는 가족은 그 지역\n으로 이주해오지 않을 것이며, 아동들을 상대로 하는 가게나 시설 역\n시 폐쇄하거나 다른 지역으로 떠날 수밖에 없다. 이와 같이 폐교는 지\n방소멸을 유발하고 가속화시키는 주요한 요인이 된다. 학생들의 사\n회성 발달, 교육행정 효율성 등에 불리한 학생희소학교의 문제를 최\n소화하기 위해 초·중 또는 중·고 통합학교나 학교 간 통합수업을 활\n성화해야 한다. 지방대학의 경쟁력을 확보하기 위해 교육부의 권한\n을 지자체에 이양하는 동시에 중앙정부 차원에서 지방대학과 산업 \n간 연계를 강화하기 위한 투자를 과감하게 늘릴 필요가 있다. 종합대\n학이 보편적인 대학 체계를 일부 단과대학 중심의 특성화 대학으로 \n전환하여 학생 

In [ ]:
question_docs = []
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

In [ ]:
question_docs

[Document(page_content='2022년 대한민국의 합계출산율은 얼마인가?', metadata={'doc_id': '9d2ac0d3-0f90-4012-bb88-ed194fb4e4cb'}),
 Document(page_content='인구가 안정적으로 유지되려면 어느 정도의 합계출산율이 필요한가?', metadata={'doc_id': '9d2ac0d3-0f90-4012-bb88-ed194fb4e4cb'}),
 Document(page_content='한국의 초저출산현상이 지속된다면 어떤 사회적 문제가 발생할 수 있는가?', metadata={'doc_id': '9d2ac0d3-0f90-4012-bb88-ed194fb4e4cb'}),
 Document(page_content='한국 사회에서 결혼과 출산의 관계는 어떻게 변화하고 있나요?', metadata={'doc_id': '37444758-5309-4f13-a4c1-d4d7482932ad'}),
 Document(page_content='자녀 양육에 대한 부담은 어떤 요소들로 인해 발생하나요?', metadata={'doc_id': '37444758-5309-4f13-a4c1-d4d7482932ad'}),
 Document(page_content='저출산 현상을 극복하기 위한 대책은 무엇인가요?', metadata={'doc_id': '37444758-5309-4f13-a4c1-d4d7482932ad'}),
 Document(page_content='서울대 아시아 센터3에서는 어떤 불합리한 차별을 없애기 위한 방안을 제시하고 있나요?', metadata={'doc_id': '078bd8ad-62a1-441f-a79d-ab035528ef64'}),
 Document(page_content='주택가격을 안정화하고, 초장기 저리 공공주택의 공급을 늘리는 이유는 무엇인가요?', metadata={'doc_id': '078bd8ad-62a1-441f-a79d-ab035528ef64'}),
 Document

In [ ]:
retriever.vectorstore.add_documents(question_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
sub_docs = vectorstore.similarity_search("한국 저출산의 원인이 뭐야?")

In [ ]:
sub_docs

[Document(page_content='저출산 현상을 극복하기 위한 대책은 무엇인가요?', metadata={'doc_id': '37444758-5309-4f13-a4c1-d4d7482932ad'}),
 Document(page_content='한국의 초저출산현상이 지속된다면 어떤 사회적 문제가 발생할 수 있는가?', metadata={'doc_id': '9d2ac0d3-0f90-4012-bb88-ed194fb4e4cb'}),
 Document(page_content='저출산 현상으로 인해 어떤 부분들이 파장을 받아야 하나요?', metadata={'doc_id': '339779dd-81a5-4d00-b495-de6cdb490103'}),
 Document(page_content='한국 사회에서 결혼과 출산의 관계는 어떻게 변화하고 있나요?', metadata={'doc_id': '37444758-5309-4f13-a4c1-d4d7482932ad'})]

In [ ]:
larger_chunks = retriever.invoke("한국 저출산의 원인이 뭐야?")
larger_chunks

[Document(page_content='세계를 잇다, 미래를 빚다!2전통적인 가부장적 유교문화의 영향이 지속되고 있어 법률에 따르지 \n않는 가족 형성은 인정되지 않고 있다. 따라서 출산은 법률혼을 전제\n로 발생한다. 그런데 한국 사회에서 결혼은 각종 조건을 충족시킬 때 \n가능하다. 학력, 학벌, 안정적 직장, 주거 등이 갖추어져야 결혼을 할 \n수 있고, 결혼 후에도 안정적인 가족생활이 가능하다. 그만큼 청년들\n은 ‘출산’에 앞서 ‘결혼’의 어려움을 직면하고 있다. 결혼의 관문을 통\n과하기 어려워지면서 만혼 경향이 증가하고, 결혼 자체를 기피하는 \n비혼도 증가하고 있다. \n과거와 달리 결혼과 출산 간 연계도 점차 약화되고 있다. 결혼이 반\n드시 출산으로 이어지지 않고 있는 것이다. 딩크족(Double Income \nNot Kids)과 같이 결혼생활이 자녀 중심에서 부부 중심으로 변화하\n는 경향도 있지만, 무엇보다도 자녀 양육에 비용이 발생하여 큰 부담\n으로 인식되고 있기 때문이다. 자녀 양육 비용은 의료·보육·(사)교육 \n등에 관한 서비스를 구매하기 위한 경제적 비용, 돌봄, 일-가정 양립 \n등을 위한 시간 비용, 출산·양육으로 인한 고용 등에서 기회비용 등\n을 포함한다. 이러한 미시적인 원인들은 개인 또는 부부가 출산을 결\n정하는 데 직접적인 영향을 미친다. 출산은 미시적인 원인들 뿐 아니\n라 거시적인 사회구조에 의해서도 영향을 받는다. 일자리 부족과 고\n용 불안정, 높은 주거비용, 과도한 교육 경쟁(학력주의와 학벌주의 만\n연), 양성 불평등, 사회규범 및 가치관의 변화 등이 출산에 영향을 미\n치는 거시적인 요소들이다. 미시적인 요인들 대부분 거시적 요인들\n에 의해 영향을 받는다. 예를 들어, 고용에서 학력주의와 학벌주의 만\n연은 사교육비를 증가시켜 자녀 양육의 경제적 부담을 높이고 있다. 출산율 회복에 성공한 국가로부터의 교훈\n이와 같이 저출산의 원인들이 명백하게 규명되고 있다는 점에서 저\n출산은 극복이 가능한 

In [ ]:
len(larger_chunks[0].page_content)

1944

In [ ]:
sub_docs = vectorstore.similarity_search("저출산으로 인한 노동력 부족에 대응하기 위해 어떤 전략들이 필요해?")
sub_docs

[Document(page_content='노동력 부족에 대응하기 위해 어떤 전략들이 필요한가요?', metadata={'doc_id': '339779dd-81a5-4d00-b495-de6cdb490103'}),
 Document(page_content='저출산 현상을 극복하기 위한 대책은 무엇인가요?', metadata={'doc_id': '37444758-5309-4f13-a4c1-d4d7482932ad'}),
 Document(page_content='출산율 회복과 인구구조 변화 대응 중 어느 것을 우선해야 하는가?', metadata={'doc_id': '75a1e36b-5f66-4668-ac40-d63e4c15677b'}),
 Document(page_content='한국의 초저출산현상이 지속된다면 어떤 사회적 문제가 발생할 수 있는가?', metadata={'doc_id': '9d2ac0d3-0f90-4012-bb88-ed194fb4e4cb'})]

In [ ]:
larger_chunks = retriever.invoke("저출산으로 인한 노동력 부족에 대응하기 위해 어떤 전략들이 필요해?")
larger_chunks[0].page_content

'세계를 잇다, 미래를 빚다!4지대를 없애도록 한다. 무엇보다도 출산과 양육에 친화적인 직장문\n화를 조성하여야 한다. \n인구구조 변화에 대응해야\n우리나라는 이미 20년 이상 동안 초저출산현상(합계출산율 1.3명 \n이하)을 겪고 있다. 적어도 당분간 ‘저출산의 덫’에서 빠져나올 수 있\n을 것으로 기대하기 어렵다. 저출산 대책들이 실효성을 거두기 시작\n할지라도 언제부터 얼마나 빠른 속도로 어느 수준까지 출산율이 높\n아질 것으로 예상하기도 어렵다. 결국 우리나라 인구는 급속하게 감\n소하고, 고령화될 것이다. 따라서 저출산 대책을 더욱 강화하는 동시\n에 인구감소 및 고령화에 대해서도 철저하게 대응하여야 한다. 인구\n감소 및 고령화가 사회에 미치는 파장은 크게 학교 붕괴, 병력자원 부\n족, 노동력 부족, 사회보장 부담 증가 등을 들 수 있다. \n학령인구 감소에 따른 학생 수 감소는 초등학교, 중학교, 고등학교 및 \n대학교에까지 영향을 미치고 있다. 과거 고출산 시기에 태어난 세대\n를 위해 건립하였던 학교 시설과 교사 양성 인프라(교육대학 등)는 과\n잉 시설이 되면서 통폐합이 진행되고 있다. 폐교는 단순하게 잉여 시\n설의 처분만을 의미하지 않는다. 폐교로 아동이 있는 가족은 그 지역\n으로 이주해오지 않을 것이며, 아동들을 상대로 하는 가게나 시설 역\n시 폐쇄하거나 다른 지역으로 떠날 수밖에 없다. 이와 같이 폐교는 지\n방소멸을 유발하고 가속화시키는 주요한 요인이 된다. 학생들의 사\n회성 발달, 교육행정 효율성 등에 불리한 학생희소학교의 문제를 최\n소화하기 위해 초·중 또는 중·고 통합학교나 학교 간 통합수업을 활\n성화해야 한다. 지방대학의 경쟁력을 확보하기 위해 교육부의 권한\n을 지자체에 이양하는 동시에 중앙정부 차원에서 지방대학과 산업 \n간 연계를 강화하기 위한 투자를 과감하게 늘릴 필요가 있다. 종합대\n학이 보편적인 대학 체계를 일부 단과대학 중심의 특성화 대학으로 \n전환하여 학생 수가 적으나 강한 대학들이 지역적으로 균